# Box Functions

## Random Boxes

These boxes are created to ensure that the top < bottom and left < right.

In [ ]:
def make_random_boxes(num_boxes=1):
    """
        It creates random boxes in YXYX format.

        Arguments:
            num_boxes: The number of boxes to create.
        
        Returns:
            A tensor of boxes with shape (num_boxes, 4) with FP values.
            All the values are in range [0., 1.]
    """
    pair_1 = tf.random.uniform((num_boxes, 2))
    pair_2 = tf.random.uniform((num_boxes, 2))

    yx_min = tf.where(pair_1 < pair_2, pair_1, pair_2)
    yx_max = tf.where(pair_1 > pair_2, pair_1, pair_2)

    boxes = tf.concat([yx_min, yx_max], axis=-1)
    return boxes

## Box Translations

Translation between YXYX, YXHW and Grid HW formats.

In [ ]:
def yx_to_indices(yx, size, dtype=tf.int64):
    return tf.cast(yx*(size - 1), dtype)

def yxyx_to_hw_grid(boxes, grid_size):
    """
        It fits a tensor of flat boxes to a grid of given size.

        Arguments:
            boxes: A tensor of boxes in YXYX format with shape (N_BOXES, 4)
            grid_size: An integer value indicating the size of the target grid.
        
        Returns:
            A grid containing heights and widths of input boxes fitted based on their top-left coordinates.
            The output shape is (grid_size, grid_size, 2)
    """
    yx_min, yx_max = tf.split(boxes, 2, axis=-1)
    hw = yx_max - yx_min

    box_indices = yx_to_indices(yx_min, grid_size, dtype=tf.int32)
    box_grid = tf.scatter_nd(indices=box_indices, updates=hw, shape=(grid_size, grid_size, 2))
    
    return box_grid

def yxkk_to_kk_grid(boxes, grid_size):
    """
        It fits a tensor of flat boxes to a grid of given size.

        Arguments:
            boxes: A tensor of boxes in YXKK format with shape (N_BOXES, 4)
            grid_size: An integer value indicating the size of the target grid.
        
        Returns:
            A grid containing heights and widths of input boxes fitted based on their top-left coordinates.
            The output shape is (grid_size, grid_size, 2)
    """
    yx_min, kk = tf.split(boxes, 2, axis=-1)

    box_indices = yx_to_indices(yx_min, grid_size, dtype=tf.int32)
    box_grid = tf.scatter_nd(indices=box_indices, updates=kk, shape=(grid_size, grid_size, 2))
    
    return box_grid

def hw_grid_to_yxhw(grid):
    """
        It translates a square HW grid to a list of boxes. Only the boxes with non-zero heights
        are qualified.

        Arguments:
            grid: A grid of of shape: (SIZE, SIZE, 2) with HW values in the last dimension.
        
        Returns:
            A tensor of shape (N_BOXES, 4). The last dimension contains YXHW values.
    """
    # Grid size
    size = grid.shape[0]

    # Generate valid box indices. We consider the boxes with non-zero heights as valid boxes
    mask = tf.cast(tf.math.not_equal(grid[:, :, 0], 0.), tf.float32)
    indices = tf.where(mask)

    # Compute YX and HW for valid boxes
    yx = tf.cast(indices/size, dtype=tf.float32)
    hw = tf.gather_nd(indices=indices, params=grid)

    # tf.print('indices: ', indices, indices.shape)
    # tf.print('first grid element', grid[indices[0][0], indices[0][1]])
    # tf.print('last grid element', grid[indices[-1][0], indices[-1][1]])

    yxhw = tf.concat([yx, hw], axis=-1)

    return yxhw

def hw_grid_to_cycxhw(grid):
    """
        It translates a grid of centers and HW values to a list of boxes.

        Arguments:
            grid: A grid of of shape: (SIZE, SIZE, 2). The last dimension contains HW values.
        
        Returns:
            A tensor of shape (N_BOXES, 4). The last dimension contains CYCXHW values.
    """
    # Grid size
    size = grid.shape[0]

    # Generate valid box indices. We consider the boxes with non-zero heights as valid boxes
    mask = tf.cast(tf.math.not_equal(grid[:, :, 0], 0.), tf.float32)
    indices = tf.where(mask)

    # Compute CYCX and HW for the boxes
    cycx = tf.cast(indices/size, dtype=tf.float32)
    hw = tf.gather_nd(indices=indices, params=grid)

    # tf.print('indices: ', indices, indices.shape)

    cycxhw = tf.concat([cycx, hw], axis=-1)

    # tf.print('cycxhw: ', cycxhw, cycxhw.shape)

    return cycxhw

def yxyx_to_cycxhw_grid(boxes, grid_size):
    """
        It converts a tensor of YXYX boxes into a grid of HW values.
        HW values are inserted based on CYCX values into the grid.

        boxes: A tensor of boxes. Shape: (N_BOXES, 4). The last dimension
            contains YXYX values.
        grid_size: The size of the grid to insert HW values.

        Returns:
            A grid with HW values. Shape: (grid_size, grid_size, 2)
    """
    # Compute box CYCX and HW.
    cy, cx = (boxes[:, 0] + boxes[:, 2])/2, (boxes[:, 1] + boxes[:, 3])/2
    cycx, hw = tf.stack([cy, cx], axis=-1), boxes[:, 2:] - boxes[:, :2]

    # Find the grid indices corresponding to the box centers.
    indices = yx_to_indices(cycx, grid_size, dtype=tf.int32)
    
    # tf.print('indices: ', indices, indices.shape)

    # Construct the grid based on the center indices and HW values
    grid = tf.scatter_nd(indices=indices, updates=hw, shape=(grid_size, grid_size, 2))

    # tf.print('grid: ', grid, grid.shape)

    return grid

def yxyx_to_yxhw(boxes):
    """
        It translates the boxes from YXYX format to YXHW format.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tensor of boxes with shape (N_BOXES, 4) in YXHW format.
    """
    yx_min, yx_max = tf.split(boxes, 2, axis=-1)
    hw = yx_max - yx_min
    hw_boxes = tf.concat([yx_min, hw], axis=-1)
    return hw_boxes

def yxhw_to_yxyx(boxes):
    """
        It translates the boxes from YXHW format to YXYX format.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXHW format.

        Returns:
            A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
    """
    yx_min, yx_hw = tf.split(boxes, 2, axis=-1)
    yx_max = tf.clip_by_value(yx_min + yx_hw, 0, 1)
    yxyx_boxes = tf.concat([yx_min, yx_max], axis=-1)
    return yxyx_boxes

def box_area(boxes):
    """
        It computes the area of the boxes.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tensor of box areas of shape (N_BOXES, 1)
    """
    y_min, x_min, y_max, x_max = tf.split(boxes, 4, axis=-1)
    return (y_max - y_min)*(x_max - x_min)

def generate_grid_indices(size, dtype=tf.int32, inverted=False):
    def cell_indices():
        indices = tf.range(size, dtype=tf.int32)
        if inverted: indices = tf.reverse(indices, axis=[0])

        grid = tf.repeat(indices, size)
        return tf.reshape(grid, (size, size))
    
    row_ids = cell_indices()
    col_ids = tf.transpose(row_ids)
    result = tf.concat([tf.expand_dims(row_ids, axis=-1), tf.expand_dims(col_ids, axis=-1)], axis=-1)
    return result

# generate_grid_indices(5, inverted=True)/4

# random_boxes = tf.map_fn(lambda v: make_random_grid_boxes(v[0], v[1]), [tf.constant([2, 3]), tf.ones((2))*4], fn_output_signature=tf.float32)

## Represenation Translations

Translations between Dense, Sparse and Ragged boxes.

In [ ]:
def sparse_to_ragged_boxes(sparse_boxes, size):
    yx_min = tf.cast(tf.reshape(sparse_boxes.indices, [-1, 6])[:, :2]/size, dtype=sparse_boxes.dtype)
    yx_max = tf.reshape(sparse_boxes.values, [-1, 2])

    # tf.print('yx_min: ', yx_min, yx_min.shape)
    # tf.print('yx_max: ', yx_max, yx_max.shape)

    boxes = tf.concat([yx_min, yx_max], axis=-1)

    return boxes

def sparsify_boxes(bboxes, size):
    bboxes = bboxes.to_tensor()
    num_boxes = tf.shape(bboxes)[0]

    # Extract box properties.
    [yx_min, yx_max] = tf.split(bboxes, 2, axis=-1)

    # Compute box dimensions which includes their heights and widths
    hw = yx_max - yx_min

    # Compute (y_min, x_min) indices of the boxes to address their top-left corner.
    yx_indices = yx_to_indices(yx_min, size)

    # Compute indices for sparse tensor of shape (IMG_SIZE, IMG_SIZE, 2) as [[Y, X, H], [Y, X, W]...].
    sparse_yx_indices = tf.repeat(yx_indices, 2, axis=0)
    sparse_hw_indices = tf.reshape(tf.repeat(tf.constant([[0, 1]], dtype=tf.int64), num_boxes, axis=0), [-1, 1])
    sparse_indices = tf.concat([sparse_yx_indices, sparse_hw_indices], axis=-1)
    sparse_values = tf.reshape(hw, [-1])

    sparse_boxes = tf.sparse.SparseTensor(indices=sparse_indices, values=sparse_values, dense_shape=(size, size, 2))

    return sparse_boxes

# boxes_yxyx_1 = tf.SparseTensor(indices=[[0, 1, 0], [0, 1, 1]], values=[.25, .5], dense_shape=[IMG_SIZE, IMG_SIZE, 2])
# boxes_yxyx_2 = tf.SparseTensor(indices=[[100, 100, 0], [100, 100, 1]], values=[.75, .95], dense_shape=[IMG_SIZE, IMG_SIZE, 2])

# sparse_to_ragged_boxes(boxes_yxyx_1, IMG_SIZE)

## Capture Neighbors

In [ ]:
def include_neighbors(indices, size=IMG_SIZE):
    """
        It adds indices of pixels surrounding the pixels in the indices.
        The results are sorted and deduped.

        Arguments:
            indices: A tensor of shape (N_ITEMS, 2) with 2-D indices to a grid of size 'size'
            size: The grid size
        
        Returns:
            A tensor of shape (N_ITEMS*8, 2) containing the indicies with neighbor indices.
            Dim 0 may be smaller than N_ITEMS*8 is there are duplicate indices.
    """
    right_reach = indices + [0, 1]
    left_reach = indices + [0, -1]
    top_reach = indices + [-1, 0]
    bottom_reach = indices + [1, 0]
    
    top_left_reach = indices + [-1, -1]
    bottom_left_reach = indices + [1, -1]
    top_right_reach = indices + [-1, 1]
    bottom_right_reach = indices + [1, 1]
    
    prospective_indices = tf.concat([
        indices,
        right_reach,
        left_reach,
        top_reach,
        bottom_reach,
        top_left_reach,
        bottom_left_reach,
        top_right_reach,
        bottom_right_reach
    ], axis=-2)

    unravelled = prospective_indices[:, 0]*size + prospective_indices[:, 1]
    sorted_indices = tf.sort(unravelled)
    unique_indices, _ = tf.unique(sorted_indices)
    # display(unique_indices)

    indices_with_neighbors = tf.transpose(tf.unravel_index(unique_indices, dims=[size, size]))
    return indices_with_neighbors

# indices = [tf.constant([[1, 3], [9, 13]]), tf.constant([[8, 3]])]

# display(indices)

# for item_indices in indices:
#     indices_with_neighbors = include_neighbors(item_indices)
#     display(indices_with_neighbors)

## Box Properties

In [ ]:
def box_area(boxes):
    """
        It computes the area of the boxes.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tensor of box areas of shape (N_BOXES, 1)
    """
    y_min, x_min, y_max, x_max = tf.split(boxes, 4, axis=-1)
    return (y_max - y_min)*(x_max - x_min)